In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pyarrow.parquet as pq
import gc
import pywt
from statsmodels.robust import mad
import scipy
from scipy import signal
from scipy.signal import butter
from tqdm import tqdm_notebook
import os
from multiprocessing import Pool

from numba import jit

import warnings

# Suppress pandas future warnings, I am using different library versions locally
# that do not raise warnings.
warnings.simplefilter(action='ignore', category=FutureWarning)

data_dir = '../input'

In [2]:
print(scipy.__version__)

1.1.0


In [3]:
# 800,000 data points taken over 20 ms
# Grid operates at 50hz, 0.02 * 50 = 1, so 800k samples in 20 milliseconds will capture one complete cycle
n_samples = 800000

# Sample duration is 20 miliseconds
sample_duration = 0.02

# Sample rate is the number of samples in one second
# Sample rate will be 40mhz
sample_rate = n_samples * (1 / sample_duration)

In [4]:
def maddest(d, axis=None):
    """
    Mean Absolute Deviation
    """
    return np.mean(np.absolute(d - np.mean(d, axis)), axis)

In [5]:
def high_pass_filter(x, low_cutoff=1000, sample_rate=sample_rate):
    """
    From @randxie https://github.com/randxie/Kaggle-VSB-Baseline/blob/master/src/utils/util_signal.py
    Modified to work with scipy version 1.1.0 which does not have the fs parameter
    """
    
    # nyquist frequency is half the sample rate https://en.wikipedia.org/wiki/Nyquist_frequency
    nyquist = 0.5 * sample_rate
    norm_low_cutoff = low_cutoff / nyquist
    
    # Fault pattern usually exists in high frequency band. According to literature, the pattern is visible above 10^4 Hz.
    # scipy version 1.2.0
    #sos = butter(10, low_freq, btype='hp', fs=sample_fs, output='sos')
    
    # scipy version 1.1.0
    sos = butter(10, Wn=[norm_low_cutoff], btype='highpass', output='sos')
    filtered_sig = signal.sosfilt(sos, x)

    return filtered_sig

In [6]:
def denoise_signal( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[1:], wavelet, mode='per' )

In [7]:
def denoise_signal_2( x, wavelet='db4', level=1):
    """
    1. Adapted from waveletSmooth function found here:
    http://connor-johnson.com/2016/01/24/using-pywavelets-to-remove-high-frequency-noise/
    2. Threshold equation and using hard mode in threshold as mentioned
    in section '3.2 denoising based on optimized singular values' from paper by Tomas Vantuch:
    http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    """
    
    # Decompose to get the wavelet coefficients
    coeff = pywt.wavedec( x, wavelet, mode="per", level=level)
    
    # Calculate sigma for threshold as defined in http://dspace.vsb.cz/bitstream/handle/10084/133114/VAN431_FEI_P1807_1801V001_2018.pdf
    # As noted by @harshit92 MAD referred to in the paper is Mean Absolute Deviation not Median Absolute Deviation
    sigma = (1/0.6745) * maddest( coeff[-level] )

    # Calculte the univeral threshold
    uthresh = sigma * np.sqrt( 2*np.log( len( x ) ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode='hard' ) for i in coeff[1:] )
    
    # Reconstruct the signal using the thresholded coefficients
    return pywt.waverec( coeff[0:], wavelet, mode='per' )

In [8]:
def remove_corona(x_dn):
    index = pd.Series(x_dn).loc[np.abs(x_dn)>0].index
    corona_idx = []
    for idx in index:
        for i in range(1,maxDistance+1):
            if idx+i < pd.Series(x_dn).shape[0]:
                if x_dn[idx+i]/(x_dn[idx]+1e-04)<-maxHeightRatio:
                    x_dn[idx:idx+maxTicksRemoval] = 0
                    corona_idx.append(idx)
    return x_dn, corona_idx

In [9]:
maxDistance = 10
maxHeightRatio = 0.25
maxTicksRemoval =500

In [10]:
subset_train = pq.read_pandas(data_dir + '/train.parquet').to_pandas()

In [11]:
numpeaks_id = np.zeros(subset_train.shape[1])
numpospeaks_id = np.zeros(subset_train.shape[1])
numnegpeaks_id = np.zeros(subset_train.shape[1])

maxamp_id = np.zeros(subset_train.shape[1])
minamp_id = np.zeros(subset_train.shape[1])
meanamp_id = np.zeros(subset_train.shape[1])

meanamppos_id = np.zeros(subset_train.shape[1])
meanampneg_id = np.zeros(subset_train.shape[1])

In [12]:
for col in tqdm_notebook(range(0, subset_train.shape[1], 3)):
    X_HP = []
    X_DN = []
    
    X_HP.append(high_pass_filter(subset_train.iloc[:,col], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(subset_train.iloc[:,col+1], low_cutoff=10000, sample_rate=sample_rate))
    X_HP.append(high_pass_filter(subset_train.iloc[:,col+2], low_cutoff=10000, sample_rate=sample_rate))
    
    X_DN.append(denoise_signal(X_HP[0], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[1], wavelet='haar', level=1))
    X_DN.append(denoise_signal(X_HP[2], wavelet='haar', level=1))
    
    X_DN[0], corona_idx = remove_corona(X_DN[0])
    X_DN[1], corona_idx = remove_corona(X_DN[1])
    X_DN[2], corona_idx = remove_corona(X_DN[2])
    
    total = X_DN[0] + X_DN[1] + X_DN[2]
    total = pd.Series(total)
    
    numpeaks_id[col:col+3] = total[total!=0].count()
    numpospeaks_id[col:col+3] = total[total>0].count()
    numnegpeaks_id[col:col+3] = total[total<0].count()
    
    meanamp_id[col:col+3] = np.mean(total)
    meanamppos_id[col:col+3] = np.mean(total[total>0])
    meanampneg_id[col:col+3] = np.mean(total[total<0])
    
    maxamp_id[col:col+3] = np.max(total)
    minamp_id[col:col+3] = np.min(total)
    

In [13]:
np.save('../features/numpeaks_id.npy', numpeaks_id)
np.save('../features/numpospeaks_id.npy', numpospeaks_id)
np.save('../features/numnegpeaks_id .npy', numnegpeaks_id )

np.save('../features/maxamp_id.npy', maxamp_id)
np.save('../features/minamp_id.npy', minamp_id )
np.save('../features/meanamp_id.npy', meanamp_id)

np.save('../features/meanamppos_id.npy', meanamppos_id)
np.save('../features/meanampneg_id.npy', meanampneg_id)